In [ ]:
#Move the file from Workspace to DBFS
dbutils.fs.cp("file:/Workspace/Shared/employeedata.csv", "dbfs:/FileStore/employeedata.csv")

True

In [ ]:
#Load CSV data into a DataFrame
df_employee = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/employeedata.csv")

#Write DataFrame to Delta format 
df_employee.write.format("delta").save("/delta/employeedata")
print("Data written to Delta format")

Data written to Delta format


In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
#Define schema for JSON file
schema = StructType([
    StructField("ProductID", IntegerType(), True), 
    StructField("ProductName", StringType(), True), 
    StructField("Category", StringType(), True), 
    StructField("Price", DoubleType(), True), 
    StructField("Stock", IntegerType(), True)
])

In [ ]:
#Move the file from Workspace to DBFS
dbutils.fs.cp("file:/Workspace/Shared/productdata.json", "dbfs:/FileStore/productdata.json")

True

In [ ]:
#Load JSON data with schema
df_product = spark.read.format("json").schema (schema).load("dbfs:/FileStore/productdata.json")
df_product.show()

+---------+-----------+-----------+------+-----+
|ProductID|ProductName|   Category| Price|Stock|
+---------+-----------+-----------+------+-----+
|      101|     Laptop|Electronics|1200.0|   35|
|      102| Smartphone|Electronics| 800.0|   80|
|      103| Desk Chair|  Furniture| 150.0|   60|
|      104|    Monitor|Electronics| 300.0|   45|
|      105|       Desk|  Furniture| 350.0|   25|
+---------+-----------+-----------+------+-----+



In [ ]:
#Create a temp view for SQL operations 
df_product.createOrReplaceTempView("product_view")

#Create a Delta table from the view
spark.sql("""
    CREATE TABLE delta_product_table
    USING DELTA
    AS SELECT * FROM product_view
""")


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1074758204289971>, line 5
      2 df_product.createOrReplaceTempView("product_view")
      4 #Create a Delta table from the view
----> 5 spark.sql("""
      6     CREATE TABLE delta_product_table
      7     USING DELTA
      8     AS SELECT * FROM product_view
      9 """)

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/session.py:1825, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1820     else:
   1821         raise PySparkTypeError(
   1822         

In [ ]:
# Move the file from Workspace to DBFS
dbutils.fs.cp("file:/Workspace/Shared/employee_updates.csv", "dbfs:/FileStore/employee_updates.csv")

True

In [ ]:
#Convert employee CSV data to Delta format
df_employee = spark.read.format("csv").option("header", "true").load("/FileStore/employee_data.csv")
df_employee.write.format("delta").mode("overwrite").save("/delta/employeedata")
print("Data written to Delta")

#Convert employee updates CSV data to Delta format
df_employee_updates = spark.read.format("csv").option("header", "true").load("/FileStore/employee_updates.csv")
df_employee_updates.write.format("delta").mode ("overwrite").save("/delta/employee_updates")
print("Data written to Delta")

Data written to Delta
Data written to Delta


In [ ]:
#Load Delta tables
df_employee = spark.read.format("delta").load("/delta/employeedata")
df_employee_updates = spark.read.format("delta").load("/delta/employee_updates")

#Create temporary views for SQL operations 
df_employee.createOrReplaceTempView("delta_employee")
df_employee_updates.createOrReplaceTempView("employee_updates")

In [ ]:
spark.sql("""
    MERGE INTO delta_employee AS target 
    USING employee_updates AS source 
    ON target.EmployeeID = source.EmployeeID 
    WHEN MATCHED THEN UPDATE SET target.Salary = source.Salary, target.Department = source.Department
    WHEN NOT MATCHED THEN INSERT (EmployeeID, Name, Department, JoiningDate, Salary) 
    VALUES (Source.EmployeeID, source.Name, source.Department, source.JoiningDate, source.Salary)
    """)

#Query the Delta table to check if the data was updated or inserted correctly 
spark.sql("SELECT * FROM delta_employee").show()

+----------+-------------+----------+-----------+------+
|EmployeeID|         Name|Department|JoiningDate|Salary|
+----------+-------------+----------+-----------+------+
|      1002|   Jane Smith|        IT| 2020-03-10| 62000|
|      1003|Emily Johnson|   Finance| 2019-07-01| 70000|
|      1004|Michael Brown|        HR| 2018-12-22| 54000|
|      1005| David Wilson|        IT| 2021-06-25| 58000|
|      1006|  Linda Davis|   Finance| 2020-11-15| 67000|
|      1007| James Miller|        IT| 2019-08-14| 65000|
|      1008|Barbara Moore|        HR| 2021-03-29| 53000|
|      1001|     John Doe|        HR| 2021-01-15| 58000|
|      1009|  Sarah Adams| Marketing| 2021-09-01| 60000|
|      1010|  Robert King|        IT| 2022-01-10| 62000|
+----------+-------------+----------+-----------+------+



In [ ]:
#write the employee dataframe to delta table
df_employee.write.format("delta").mode("overwrite").save("/delta/employeedata")

In [ ]:
#register the delta table
spark.sql("create table if not exists delta_employee using delta location '/delta/employeedata'")

DataFrame[]

In [ ]:
#optimize the delta table
spark.sql("OPTIMIZE delta_employee")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2439523593528616>, line 2
      1 #optimize the delta table
----> 2 spark.sql("OPTIMIZE delta_employee")

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/session.py:1825, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1820     else:
   1821         raise PySparkTypeError(
   1822             error_class="INVALID_TYPE",
   1823             message_parameters={"arg_name": "args", "arg_type": type(args).__name__},
   1824         )
-> 1825     return DataFra

In [ ]:
spark.catalog.tableExists("delta_employee")

True

In [ ]:
# Register the delta table
spark.sql("CREATE TABLE IF NOT EXISTS delta_employee USING DELTA LOCATION '/delta/employeedata'")

# Check if the table exists
table_exists = spark.catalog.tableExists("delta_employee")
print(f"Table exists: {table_exists}")

if table_exists:
    # Convert the view to a table
    spark.sql("CREATE OR REPLACE TEMPORARY VIEW delta_employee_view AS SELECT * FROM delta_employee")
    print("View converted to table.")

    # Optimize the delta table
    spark.sql("OPTIMIZE delta.`/delta/employeedata`")
    print("Optimization completed.")
else:
    print("Table delta_employee does not exist.")

Table exists: True
View converted to table.
Optimization completed.


In [ ]:
#describe the history of the delta
#spark.sql("DESCRIBE HISTORY delta_employee").show(truncate=False)
spark.sql("DESCRIBE HISTORY delta.`/delta/employeedata`").show(truncate=False)

+-------+-------------------+----------------+----------------------------------+---------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+------------------+--------------------+-----------+-----------------+-------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
spark.sql("""
    OPTIMIZE delta.`/delta/employeedata` ZORDER BY Department
""")

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

In [ ]:
spark.sql("""
    VACUUM delta.`/delta/employeedata` RETAIN 168 HOURS 
""")

DataFrame[path: string]

In [ ]:
%sql
CREATE TABLE managed_table(
  id INT,
  name STRING
);

In [ ]:
%sql
CREATE EXTERNAL TABLE unmanaged_table(
  id INT,
  name STRING 

)
LOCATION '/user/data/external_data/';